In [8]:
!pip install lightgbm

In [9]:
!git clone https://github.com/Keshfer/Chocolate-Rating-ML

Cloning into 'Chocolate-Rating-ML'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 100 (delta 53), reused 62 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (100/100), 3.44 MiB | 8.62 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [10]:
cd Chocolate-Rating-ML

/content/Chocolate-Rating-ML/Chocolate-Rating-ML


In [11]:
# Import approriate functions and packages
import numpy as np
import pandas as pd
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Import the dataframe
df = pd.read_csv('cacao_engineered.csv')

# Create X and y training and testing data
X = df.drop(['Rating'], axis=1)
y = df['Rating']

# Clean up columns of X so it works with the model
X.columns = [col.replace(" ", "_").replace(":", "_") for col in X.columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [26]:
# Start creating lgb regression model
# Create the lgb Dataset objects to store the training and testing data
train_data = lgb.Dataset(X_train, label = y_train)
test_data = lgb.Dataset(X_test, label = y_test, reference=train_data)

# Set parameters for the lgb regression model
# Objective: Set to regression so that the model tries to predict a regressive value (in this case, quality)
# Metric: The loss metric to be used for the model. Set to mse for now.
# Learning rate: How much to update the trees when refining
# Feature fraction: The percentage of features to take into account when making model, used to prevent overfitting
params = {
    'objective': 'regression',
    'metric': 'mse',
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'verbose':-1
}

# Train the actual model, passes in the parameters, training data, the number of rounds to train
num_round = 100  # Number of boosting rounds
model = lgb.train(
    params,
    train_data,
    num_boost_round=num_round,
    valid_sets=[train_data, test_data],
    valid_names=['train', 'eval'],
)

# Predict on the test set, using the best iteration of the model for training
y_pred = model.predict(X_test, num_iteration = model.best_iteration)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
print(f"MSE: {mse}")

MSE: 0.19547680507452356


In [29]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

# Create the model
model = LGBMRegressor()

# Parameter grid for grid search algorithm
param_grid = {
    'objective': ['regression'],
    'metric': ['mse'],
    'lambda_l2': [0.1],
    'feature_fraction': [0.8, 0.9, 1.0],
    'num_leaves': [20, 31, 40],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [30, 50, 100],
    'max_depth': [-1, 10, 20]
}

# Grid search
grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=3
)

# Fit the grid search
grid.fit(X_train, y_train)

# Print best parameters and score
print("Best Parameters:", grid.best_params_)
print("Best Score:", grid.best_score_)

# Evaluate on test data
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Test MSE:", mse)

Best Parameters: {'feature_fraction': 0.8, 'lambda_l2': 0.1, 'learning_rate': 0.05, 'max_depth': 20, 'metric': 'mse', 'n_estimators': 50, 'num_leaves': 40, 'objective': 'regression'}
Best Score: 0.11746596541815173
Test MSE: 0.19290720697219907
